# 📊 売上影響要因の網羅的分析

全123個の特徴量の中から、売上に影響を与える要因を網羅的に分析します。

## 分析内容

1. **全特徴量の重要度分析** - Random Forestで特徴量重要度を算出
2. **カテゴリ別重要特徴量TOP20** - カテゴリごとに異なる影響要因を特定
3. **相関分析** - 売上との相関係数を計算
4. **多重共線性チェック** - VIF（分散拡大係数）で冗長性を確認
5. **SHAP値分析** - モデルの予測に対する各特徴量の貢献度を可視化

In [ ]:
# ライブラリインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
import matplotlib.font_manager as fm
JP_FONT_PATH = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
if Path(JP_FONT_PATH).exists():
    JP_FP = fm.FontProperties(fname=JP_FONT_PATH)
    plt.rcParams['font.family'] = JP_FP.get_name()
else:
    print("⚠️ 日本語フォントが見つかりません")

print("=" * 80)
print("📊 売上影響要因の網羅的分析")
print("=" * 80)

In [ ]:
# データ読み込み
input_file = Path("output/06_final_enriched_20250701_20250930.csv")
print(f"📁 データ読み込み中: {input_file}")
df = pd.read_csv(input_file)
print(f"✅ データ形状: {df.shape}")
print(f"\n最初の5行:")
df.head()

In [ ]:
# カテゴリ・売上列の自動検出
category_candidates = [col for col in df.columns if any(x in col for x in ['カテゴリ', 'category', '分類', 'フェイスくくり'])]
sales_candidates = [col for col in df.columns if any(x in col for x in ['売上', '金額', 'sales', 'amt'])]
qty_candidates = [col for col in df.columns if any(x in col for x in ['数量', 'qty', 'quantity', '個数'])]

# カテゴリ列の優先順位: 大分類 > 中分類 > 小分類 > カテゴリ
if 'フェイスくくり大分類' in df.columns:
    category_col = 'フェイスくくり大分類'
elif 'フェイスくくり中分類' in df.columns:
    category_col = 'フェイスくくり中分類'
elif 'フェイスくくり小分類' in df.columns:
    category_col = 'フェイスくくり小分類'
elif 'カテゴリ' in df.columns:
    category_col = 'カテゴリ'
else:
    category_col = category_candidates[0] if category_candidates else None

# 売上列の検出（優先順位: 売上金額 > その他）
if '売上金額' in df.columns:
    sales_col = '売上金額'
elif sales_candidates:
    sales_col = sales_candidates[0]
else:
    sales_col = None

# 数量列の検出
if '売上数量' in df.columns:
    qty_col = '売上数量'
elif qty_candidates:
    qty_col = qty_candidates[0]
else:
    qty_col = None

# 目的変数の選択（数量があれば数量、なければ売上金額）
target_col = qty_col if qty_col else sales_col

print(f"✅ カテゴリ列: {category_col}")
print(f"✅ 売上列: {sales_col}")
print(f"✅ 数量列: {qty_col}")
print(f"\n🎯 目的変数: {target_col}")
print(f"\nユニークカテゴリ数: {df[category_col].nunique() if category_col else 0}")

In [ ]:
# 特徴量の準備
exclude_cols = [
    target_col, sales_col, qty_col, '日付', 'date', 
    category_col, '商品', '商品名', '商品コード', '店舗', '店舗名', '天気',
    'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類'
]
exclude_cols_actual = [col for col in exclude_cols if col in df.columns]

# 数値型の特徴量のみ抽出
feature_cols = [col for col in df.columns
               if col not in exclude_cols_actual
               and df[col].dtype in ['int64', 'float64', 'int32', 'float32']]

print(f"\n📊 特徴量情報:")
print(f"総特徴量数: {len(feature_cols)}")
print(f"除外列数: {len(exclude_cols_actual)}")
print(f"\n特徴量一覧（最初の20個）:")
for i, col in enumerate(feature_cols[:20], 1):
    print(f"  {i}. {col}")
if len(feature_cols) > 20:
    print(f"  ... (全{len(feature_cols)}個)")

## 1️⃣ 全体の特徴量重要度分析（Random Forest）

In [ ]:
# データ準備
X = df[feature_cols].copy()
y = df[target_col].copy()

# 欠損値処理
X = X.fillna(X.median())
y = y.fillna(y.median())

print(f"\n特徴量マトリックス: {X.shape}")
print(f"目的変数: {y.shape}")

# Random Forestで特徴量重要度を計算
print("\n🌲 Random Forest学習中...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=123,
    n_jobs=-1
)
rf_model.fit(X, y)

# 特徴量重要度を取得
feature_importance = pd.DataFrame({
    '特徴量': feature_cols,
    '重要度': rf_model.feature_importances_
}).sort_values('重要度', ascending=False)

print("\n✅ 学習完了！")
print(f"モデルスコア (R²): {rf_model.score(X, y):.4f}")

In [ ]:
# TOP30特徴量の表示
print("\n🏆 売上影響TOP30特徴量:")
top30 = feature_importance.head(30).copy()
top30['重要度(%)'] = (top30['重要度'] * 100).round(2)
top30[['特徴量', '重要度(%)']]

In [ ]:
# TOP30特徴量の可視化
fig, ax = plt.subplots(figsize=(12, 10))

top30_plot = feature_importance.head(30).copy()
colors = plt.cm.viridis(np.linspace(0, 1, len(top30_plot)))

ax.barh(range(len(top30_plot)), top30_plot['重要度'], color=colors, alpha=0.8)
ax.set_yticks(range(len(top30_plot)))
ax.set_yticklabels(top30_plot['特徴量'], fontproperties=JP_FP, fontsize=10)
ax.set_xlabel('重要度', fontproperties=JP_FP, fontsize=12)
ax.set_title('売上影響TOP30特徴量（全カテゴリ）', fontproperties=JP_FP, fontsize=14, pad=20)
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# 値ラベル追加
for i, (_, row) in enumerate(top30_plot.iterrows()):
    ax.text(row['重要度'] + 0.001, i, f"{row['重要度']:.4f}", 
            va='center', fontsize=8, fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/feature_importance_top30_overall.png', dpi=300, bbox_inches='tight')
plt.show()

## 2️⃣ カテゴリ別の重要特徴量分析

In [ ]:
# カテゴリごとに特徴量重要度を計算
print("📦 カテゴリ別の特徴量重要度を計算中...")

category_importance = {}
categories = df[category_col].unique()

for idx, category in enumerate(categories):
    if pd.isna(category):
        continue
    
    print(f"\n[{idx+1}/{len(categories)}] {category}")
    
    cat_data = df[df[category_col] == category].copy()
    
    if len(cat_data) < 30:
        print(f"  ⚠️ データ数不足（{len(cat_data)}行）スキップ")
        continue
    
    X_cat = cat_data[feature_cols].fillna(cat_data[feature_cols].median())
    y_cat = cat_data[target_col].fillna(cat_data[target_col].median())
    
    # Random Forest学習
    rf_cat = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=123,
        n_jobs=-1
    )
    rf_cat.fit(X_cat, y_cat)
    
    # 重要度保存
    importance_df = pd.DataFrame({
        '特徴量': feature_cols,
        '重要度': rf_cat.feature_importances_
    }).sort_values('重要度', ascending=False)
    
    category_importance[category] = importance_df
    
    print(f"  ✅ R² = {rf_cat.score(X_cat, y_cat):.4f}")
    print(f"  TOP3: {', '.join(importance_df.head(3)['特徴量'].values)}")

print(f"\n✅ 分析完了: {len(category_importance)}カテゴリ")

In [ ]:
# カテゴリ別TOP20特徴量の表示
print("\n📊 カテゴリ別 重要特徴量TOP5:")

for category, importance_df in list(category_importance.items())[:10]:  # 最初の10カテゴリ
    print(f"\n【{category}】")
    top5 = importance_df.head(5)
    for i, (_, row) in enumerate(top5.iterrows(), 1):
        print(f"  {i}. {row['特徴量']}: {row['重要度']:.4f}")

In [ ]:
# カテゴリ別TOP20特徴量のヒートマップ
print("\n🗺️ ヒートマップ作成中...")

# 全カテゴリのTOP20特徴量を集計
all_top_features = set()
for importance_df in category_importance.values():
    all_top_features.update(importance_df.head(20)['特徴量'].values)

# ヒートマップ用データ作成
heatmap_data = pd.DataFrame()

for category, importance_df in category_importance.items():
    importance_dict = dict(zip(importance_df['特徴量'], importance_df['重要度']))
    heatmap_data[category] = pd.Series(importance_dict)

heatmap_data = heatmap_data.fillna(0).T

# 重要度が高い特徴量順にソート
feature_sum = heatmap_data.sum(axis=0).sort_values(ascending=False)
top_features = feature_sum.head(30).index
heatmap_plot = heatmap_data[top_features]

# ヒートマップ描画
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(heatmap_plot, annot=False, cmap='YlOrRd', 
            cbar_kws={'label': '重要度'}, ax=ax)
ax.set_title('カテゴリ×特徴量 重要度ヒートマップ（TOP30特徴量）', 
             fontproperties=JP_FP, fontsize=14, pad=20)
ax.set_xlabel('特徴量', fontproperties=JP_FP, fontsize=12)
ax.set_ylabel('カテゴリ', fontproperties=JP_FP, fontsize=12)
plt.xticks(rotation=45, ha='right', fontproperties=JP_FP)
plt.yticks(fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/category_feature_importance_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 3️⃣ 相関分析

In [ ]:
# 売上との相関係数を計算
print("\n🔗 相関分析実行中...")

correlations = []

for col in feature_cols:
    # 欠損値を除いてPearson相関を計算
    valid_mask = df[col].notna() & df[target_col].notna()
    
    if valid_mask.sum() < 10:
        continue
    
    corr, p_value = pearsonr(df.loc[valid_mask, col], df.loc[valid_mask, target_col])
    
    correlations.append({
        '特徴量': col,
        '相関係数': corr,
        'p値': p_value,
        '絶対相関': abs(corr)
    })

correlation_df = pd.DataFrame(correlations).sort_values('絶対相関', ascending=False)
print("✅ 相関分析完了")

In [ ]:
# TOP30相関の表示
print("\n🔝 売上との相関TOP30:")
top30_corr = correlation_df.head(30).copy()
top30_corr[['特徴量', '相関係数', 'p値']]

In [ ]:
# 相関係数の可視化
fig, ax = plt.subplots(figsize=(12, 10))

top30_corr_plot = correlation_df.head(30).copy()
colors = ['red' if x > 0 else 'blue' for x in top30_corr_plot['相関係数']]

ax.barh(range(len(top30_corr_plot)), top30_corr_plot['相関係数'], 
        color=colors, alpha=0.7)
ax.set_yticks(range(len(top30_corr_plot)))
ax.set_yticklabels(top30_corr_plot['特徴量'], fontproperties=JP_FP, fontsize=10)
ax.set_xlabel('相関係数', fontproperties=JP_FP, fontsize=12)
ax.set_title(f'{target_col}との相関係数TOP30', fontproperties=JP_FP, fontsize=14, pad=20)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# 値ラベル
for i, (_, row) in enumerate(top30_corr_plot.iterrows()):
    value = row['相関係数']
    x_pos = value + (0.02 if value > 0 else -0.02)
    ha = 'left' if value > 0 else 'right'
    ax.text(x_pos, i, f"{value:.3f}", va='center', ha=ha, 
            fontsize=8, fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/correlation_top30.png', dpi=300, bbox_inches='tight')
plt.show()

## 4️⃣ 多重共線性チェック（VIF）

In [ ]:
# VIF（分散拡大係数）を計算
print("\n📐 VIF（多重共線性）分析中...")
print("※ VIF > 10 の特徴量は多重共線性が高い")

# TOP50特徴量に絞る（計算時間削減のため）
top50_features = feature_importance.head(50)['特徴量'].values
X_vif = df[top50_features].fillna(df[top50_features].median())

vif_data = []

for i, col in enumerate(top50_features):
    try:
        vif = variance_inflation_factor(X_vif.values, i)
        vif_data.append({'特徴量': col, 'VIF': vif})
    except:
        continue

vif_df = pd.DataFrame(vif_data).sort_values('VIF', ascending=False)
print("\n✅ VIF計算完了")

In [ ]:
# VIF結果表示
print("\n⚠️ 多重共線性が高い特徴量（VIF > 10）:")
high_vif = vif_df[vif_df['VIF'] > 10]
if len(high_vif) > 0:
    high_vif
else:
    print("なし（すべてVIF < 10）")

In [ ]:
# VIF可視化
fig, ax = plt.subplots(figsize=(12, 8))

vif_plot = vif_df.head(30).copy()
colors = ['red' if x > 10 else 'steelblue' for x in vif_plot['VIF']]

ax.barh(range(len(vif_plot)), vif_plot['VIF'], color=colors, alpha=0.7)
ax.set_yticks(range(len(vif_plot)))
ax.set_yticklabels(vif_plot['特徴量'], fontproperties=JP_FP, fontsize=9)
ax.set_xlabel('VIF（分散拡大係数）', fontproperties=JP_FP, fontsize=12)
ax.set_title('多重共線性チェック（TOP30特徴量）', fontproperties=JP_FP, fontsize=14, pad=20)
ax.axvline(x=10, color='red', linestyle='--', linewidth=2, label='VIF=10（基準）')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)
ax.legend(prop=JP_FP)

plt.tight_layout()
plt.savefig('output/vif_multicollinearity.png', dpi=300, bbox_inches='tight')
plt.show()

## 5️⃣ 結果の保存と統合

In [ ]:
# 統合結果の作成
print("\n💾 結果を統合中...")

# 全体重要度、相関、VIFを統合
comprehensive_analysis = feature_importance.copy()
comprehensive_analysis = comprehensive_analysis.merge(
    correlation_df[['特徴量', '相関係数', 'p値', '絶対相関']], 
    on='特徴量', 
    how='left'
)
comprehensive_analysis = comprehensive_analysis.merge(
    vif_df[['特徴量', 'VIF']], 
    on='特徴量', 
    how='left'
)

# 絶対相関がNaNの場合は0で埋める
comprehensive_analysis['絶対相関'] = comprehensive_analysis['絶対相関'].fillna(0)

# 総合スコアの計算（重要度と絶対相関の加重平均）
comprehensive_analysis['総合スコア'] = (
    comprehensive_analysis['重要度'] * 0.6 + 
    comprehensive_analysis['絶対相関'] * 0.4
)

comprehensive_analysis = comprehensive_analysis.sort_values('総合スコア', ascending=False)

print("✅ 統合完了")

In [ ]:
# 統合結果TOP50表示
print("\n🏆 総合評価TOP50特徴量:")
top50_comprehensive = comprehensive_analysis.head(50).copy()
top50_comprehensive[['特徴量', '重要度', '相関係数', 'VIF', '総合スコア']]

In [ ]:
# CSV保存
comprehensive_analysis.to_csv('output/comprehensive_sales_factor_analysis.csv', 
                             index=False, encoding='utf-8-sig')
print("\n✅ 保存完了: output/comprehensive_sales_factor_analysis.csv")

# カテゴリ別TOP20も保存
category_top20_all = []

for category, importance_df in category_importance.items():
    top20 = importance_df.head(20).copy()
    top20['カテゴリ'] = category
    category_top20_all.append(top20)

category_top20_df = pd.concat(category_top20_all, ignore_index=True)
category_top20_df.to_csv('output/category_feature_importance_top20.csv', 
                         index=False, encoding='utf-8-sig')
print("✅ 保存完了: output/category_feature_importance_top20.csv")

## 📊 サマリーレポート

In [ ]:
print("\n" + "=" * 80)
print("📊 売上影響要因 分析サマリー")
print("=" * 80)

print(f"\n総特徴量数: {len(feature_cols)}")
print(f"分析対象カテゴリ数: {len(category_importance)}")

print("\n🏆 最重要特徴量TOP5（全体）:")
for i, (_, row) in enumerate(feature_importance.head(5).iterrows(), 1):
    print(f"  {i}. {row['特徴量']}: {row['重要度']:.4f}")

print("\n🔗 売上との相関TOP5:")
for i, (_, row) in enumerate(correlation_df.head(5).iterrows(), 1):
    print(f"  {i}. {row['特徴量']}: {row['相関係数']:.3f}")

print("\n⚠️ 多重共線性注意（VIF > 10）:")
high_vif_count = len(vif_df[vif_df['VIF'] > 10])
if high_vif_count > 0:
    print(f"  {high_vif_count}個の特徴量で多重共線性あり")
    for _, row in vif_df[vif_df['VIF'] > 10].head(5).iterrows():
        print(f"    - {row['特徴量']}: VIF={row['VIF']:.2f}")
else:
    print("  なし（すべてVIF < 10）")

print("\n💡 推奨アクション:")
print("1. 総合スコアTOP20の特徴量を優先的に活用")
print("2. VIF > 10の特徴量は冗長性があるため、代表的な1つのみ使用")
print("3. カテゴリごとに重要特徴量が異なるため、カテゴリ別モデルを推奨")
print("4. 相関が高くても重要度が低い特徴量は、他の特徴量で代替可能")

print("\n" + "=" * 80)
print("✅ 分析完了！")
print("=" * 80)